In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Define transformations
class CocoTransform:
    def __call__(self, image, target):
        image = F.to_tensor(image)  # Convert PIL image to tensor
        return image, target

In [3]:
# Dataset class
def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )

# Load datasets
train_dataset = get_coco_dataset(
    img_dir=r"D:\AI stuff\CV-LEC-FinalProject\RCNN\Construction Site Safety.v27-yolov8s.coco\train",
    ann_file=r"D:\AI stuff\CV-LEC-FinalProject\RCNN\Construction Site Safety.v27-yolov8s.coco\train\_annotations.coco.json"
)


val_dataset = get_coco_dataset(
    img_dir=r"D:\AI stuff\CV-LEC-FinalProject\RCNN\Construction Site Safety.v27-yolov8s.coco\valid",
    ann_file=r"D:\AI stuff\CV-LEC-FinalProject\RCNN\Construction Site Safety.v27-yolov8s.coco\valid\_annotations.coco.json"
)



# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
# Load Faster R-CNN with ResNet-50 backbone
def get_model(num_classes):
    # Load pre-trained Faster R-CNN
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [5]:
# Initialize the model
num_classes = 11 # Background + chair, human, table
model = get_model(num_classes)

c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer and learning rate scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [7]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    for images, targets in data_loader:
        # Move images to the device
        images = [img.to(device) for img in images]

        # Validate and process targets
        processed_targets = []
        valid_images = []
        for i, target in enumerate(targets):
            boxes = []
            labels = []
            for obj in target:
                # Extract bbox
                bbox = obj["bbox"]  # Format: [x, y, width, height]
                x, y, w, h = bbox

                # Ensure the width and height are positive
                if w > 0 and h > 0:
                    boxes.append([x, y, x + w, y + h])  # Convert to [x_min, y_min, x_max, y_max]
                    labels.append(obj["category_id"])

            # Only process if there are valid boxes
            if boxes:
                processed_target = {
                    "boxes": torch.tensor(boxes, dtype=torch.float32).to(device),
                    "labels": torch.tensor(labels, dtype=torch.int64).to(device),
                }
                processed_targets.append(processed_target)
                valid_images.append(images[i])  # Add only valid images

        # Skip iteration if no valid targets
        if not processed_targets:
            continue

        # Ensure images and targets are aligned
        images = valid_images

        # Forward pass
        loss_dict = model(images, processed_targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch [{epoch}] Loss: {losses.item():.4f}")

In [8]:
# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch)
    lr_scheduler.step()

    # Save the model's state dictionary after every epoch
    model_path = f"fasterrcnn_resnet50_epoch_{epoch + 1}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved: {model_path}")

Epoch [0] Loss: 1.1743
Model saved: fasterrcnn_resnet50_epoch_1.pth
Epoch [1] Loss: 0.8638
Model saved: fasterrcnn_resnet50_epoch_2.pth
Epoch [2] Loss: 1.0199
Model saved: fasterrcnn_resnet50_epoch_3.pth
Epoch [3] Loss: 0.9162
Model saved: fasterrcnn_resnet50_epoch_4.pth
Epoch [4] Loss: 0.5871
Model saved: fasterrcnn_resnet50_epoch_5.pth
Epoch [5] Loss: 0.5349
Model saved: fasterrcnn_resnet50_epoch_6.pth
Epoch [6] Loss: 0.4632
Model saved: fasterrcnn_resnet50_epoch_7.pth
Epoch [7] Loss: 0.5958
Model saved: fasterrcnn_resnet50_epoch_8.pth
Epoch [8] Loss: 0.5480
Model saved: fasterrcnn_resnet50_epoch_9.pth
Epoch [9] Loss: 0.6748
Model saved: fasterrcnn_resnet50_epoch_10.pth
Epoch [10] Loss: 0.6173
Model saved: fasterrcnn_resnet50_epoch_11.pth
Epoch [11] Loss: 0.7832
Model saved: fasterrcnn_resnet50_epoch_12.pth
Epoch [12] Loss: 0.7608
Model saved: fasterrcnn_resnet50_epoch_13.pth
Epoch [13] Loss: 0.6743
Model saved: fasterrcnn_resnet50_epoch_14.pth
Epoch [14] Loss: 1.0043
Model saved: fa